In [1]:
import urllib.request
from bs4 import BeautifulSoup
import os
from joblib import Parallel, delayed
import multiprocessing
from time import time
import requests 
import urllib.request

In [2]:
def get_text_html(link):
    return BeautifulSoup(requests.get(url=link).text, 'html.parser')

In [3]:
url = "https://www.gutenberg.org/browse/authors/a"

In [4]:
page = get_text_html(url)

In [5]:
def get_pages_by_author(data):
    pages = {}
    name_author = ''
    for item in data:
        tag_name = item.name
        if tag_name == 'h2':
            name_author = item.text.replace(' ¶','')
#             print(name_author)
        elif tag_name == 'ul':
            name_book = item.text
            links_book = []
            for i in item.findChildren('li', recurdive=False):
                if 'English' in i.text:
                    links_on_book = i.find_all('a')
                    for link in links_on_book:
                        if 'ebooks' in link['href']:
                            links_book.append(f"https://www.gutenberg.org{link['href']}")
            if len(links_book) > 0:
                pages[name_author] = links_book
    return pages

In [53]:
def get_direct_reference(link):
    result = 'empty'
    page = get_text_html(link)
    data = page.find("table", {"class": "files"}).findChildren('tr')
    for i in data:
        attributes = i.attrs
        if 'about' in attributes:
            if 'txt' in i['about']:
                result = i['about']
    return result

In [54]:
def download_by_link(link):
    urllib.request.urlretrieve(keu_for_download, dir_name + '/' + link[key])

In [55]:
pages = get_pages_by_author(page.find("div", {"class": "pgdbbyauthor"}).find('div').children)

In [56]:
def get_linc_from_dict(pages):
    count = 1
    for key in pages.keys():
        a_pool = multiprocessing.Pool()
        print('Key: ', pages[key])
        res = a_pool.map(get_direct_reference, pages[key])
        print(res)
        if count <= 7:
            count += 1
        else:
            break
#         for page_book in pages[key]:
#             get_direct_reference(page_book)

In [57]:
get_linc_from_dict(pages)

Key:  ['https://www.gutenberg.org/ebooks/29666']
['https://www.gutenberg.org/ebooks/29666.txt.utf-8']
Key:  ['https://www.gutenberg.org/ebooks/61217']
['https://www.gutenberg.org/ebooks/61217.txt.utf-8']
Key:  ['https://www.gutenberg.org/ebooks/26348']
['https://www.gutenberg.org/ebooks/26348.txt.utf-8']
Key:  ['https://www.gutenberg.org/ebooks/11861', 'https://www.gutenberg.org/ebooks/25753']
['https://www.gutenberg.org/ebooks/11861.txt.utf-8', 'https://www.gutenberg.org/ebooks/25753.txt.utf-8']
Key:  ['https://www.gutenberg.org/ebooks/10338']
['https://www.gutenberg.org/ebooks/10338.txt.utf-8']
Key:  ['https://www.gutenberg.org/ebooks/16791']
['https://www.gutenberg.org/ebooks/16791.txt.utf-8']
Key:  ['https://www.gutenberg.org/ebooks/67167', 'https://www.gutenberg.org/ebooks/62187']
['https://www.gutenberg.org/ebooks/67167.txt.utf-8', 'https://www.gutenberg.org/files/62187/62187-0.txt']
Key:  ['https://www.gutenberg.org/ebooks/23037']
['https://www.gutenberg.org/ebooks/23037.txt.utf

In [58]:
print(get_direct_reference('https://www.gutenberg.org/ebooks/62187'))

https://www.gutenberg.org/files/62187/62187-0.txt


In [7]:
def get_list_link(page):
    links = []
    pgdbbyauthor = page.find("div", {"class": "pgdbbyauthor"})
    for element in pgdbbyauthor.children:
        if element.name == 'h2':
            pass
        if element.name == 'ul':
            for link in element.find_all('a'):
                link_key = link['href']
                if 'ebooks' in link_key:
                    links.append({'https://www.gutenberg.org' + link_key: link.text})
    return links

In [10]:
links

[]

In [11]:
def create_file(text, path):
    if os.path.isfile(path) == False:
        with open(path + '.txt', 'w') as file: 
            file.write(text) 

In [12]:
def create_dir(name):
    if os.path.exists(name) == False:
        os.mkdir(name)

In [13]:
def get_link_for_download(url):
    content = get_content(url)
    elements = BeautifulSoup(content, 'html.parser').find('table').find_all('tr')
    for element in elements:
        for attribute in element.attrs:
            if attribute == 'about':
                if '.txt' in element['about']:
                    return element['about']

In [14]:
def create_file_by_linc(link, dir_name):
    key = list(link.keys())[0]
    keu_for_download = get_link_for_download(key)
    r = requests.get(keu_for_download, stream=True)
    if r.status_code == 200:
        with open(dir_name + '/' + link[key], 'wb') as f:
            for chunk in r:
                f.write(chunk)
    #urllib.request.urlretrieve(keu_for_download, dir_name + '/' + link[key])

In [15]:
def download_files(links):
    dir_name = 'A'
    create_dir(dir_name)
    for link in links:
        create_file_by_linc(link, dir_name)
    #ThreadPool(9).imap_unordered(create_file_by_linc, links)

In [16]:
download_files(links)